In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("/home/lcz/lenlp/text-generation/py_project")

In [55]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [5]:
from utils.data_loader import load_dataset
from utils.data_loader import load_test_dataset
from utils.linux_config import embedding_matrix_path

root path: /home/lcz/lenlp/text-generation


In [20]:
train_X, train_Y, test_X = load_dataset()

In [8]:
from utils.wv_loader import get_embedding_matrix
wv_model_path = "./../data/word2vec.model"
embedding_matrix = get_embedding_matrix(wv_model_path)
import numpy as np
np.save("./../data/embedding_matrix.npy", embedding_matrix)

2020-05-24 08:46:09,463 : INFO : loading Word2Vec object from ./../data/word2vec.model
2020-05-24 08:46:11,903 : INFO : loading wv recursively from ./../data/word2vec.model.wv.* with mmap=None
2020-05-24 08:46:11,908 : INFO : setting ignored attribute vectors_norm to None
2020-05-24 08:46:11,913 : INFO : loading vocabulary recursively from ./../data/word2vec.model.vocabulary.* with mmap=None
2020-05-24 08:46:11,914 : INFO : loading trainables recursively from ./../data/word2vec.model.trainables.* with mmap=None
2020-05-24 08:46:11,915 : INFO : setting ignored attribute cum_table to None
2020-05-24 08:46:11,917 : INFO : loaded ./../data/word2vec.model


In [9]:
embedding_matrix

array([[ 5.78204274e-01,  6.03120551e-02,  3.60607475e-01, ...,
        -3.83604765e-01, -2.35126719e-01, -3.39127541e-01],
       [ 2.48232469e-01, -1.09330297e-01, -1.07853085e-01, ...,
        -4.05686855e-01, -3.84583443e-01,  8.72177184e-01],
       [-2.61794060e-01,  1.58479378e-01, -3.25242400e-01, ...,
        -1.04319692e+00, -3.70189011e-01, -6.24906011e-02],
       ...,
       [ 3.13954473e-01,  6.27483010e-01,  8.70058371e-04, ...,
        -6.59883201e-01,  1.04633875e-01,  5.38641512e-02],
       [-3.03027540e-01, -6.11446559e-01, -1.08515084e+00, ...,
         1.52157664e-01,  3.57411057e-01, -8.81655991e-01],
       [ 3.11857313e-01, -5.35719246e-02, -3.20000172e-01, ...,
        -1.04822442e-01,  1.92798018e-01, -4.42412287e-01]], dtype=float32)

In [14]:
from utils.wv_loader import load_embedding_matrix, load_vocab
embedding_matrix2 = load_embedding_matrix("./../data/embedding_matrix")

In [15]:
embedding_matrix2

array([[ 5.78204274e-01,  6.03120551e-02,  3.60607475e-01, ...,
        -3.83604765e-01, -2.35126719e-01, -3.39127541e-01],
       [ 2.48232469e-01, -1.09330297e-01, -1.07853085e-01, ...,
        -4.05686855e-01, -3.84583443e-01,  8.72177184e-01],
       [-2.61794060e-01,  1.58479378e-01, -3.25242400e-01, ...,
        -1.04319692e+00, -3.70189011e-01, -6.24906011e-02],
       ...,
       [ 3.13954473e-01,  6.27483010e-01,  8.70058371e-04, ...,
        -6.59883201e-01,  1.04633875e-01,  5.38641512e-02],
       [-3.03027540e-01, -6.11446559e-01, -1.08515084e+00, ...,
         1.52157664e-01,  3.57411057e-01, -8.81655991e-01],
       [ 3.11857313e-01, -5.35719246e-02, -3.20000172e-01, ...,
        -1.04822442e-01,  1.92798018e-01, -4.42412287e-01]], dtype=float32)

In [16]:
vocab_path = "./../data/vocab.txt"
vocab, reverse_vocab = load_vocab(vocab_path)

In [17]:
len(vocab)

32909

In [44]:
units = 1024
params = {}
params["vocab_size"] = len(vocab)
params["embed_size"] = 300
params["enc_units"] = units
params["attn_units"] = units
params["dec_units"] = units
params["batch_size"] = 64
params["epochs"] = 2
params["max_enc_len"] = 200
params["max_dec_len"] = 41

## 构建数据集

In [48]:
import tensorflow as tf
steps_per_epoch = len(train_X) // params["batch_size"]
dataset = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).shuffle(params["batch_size"])
dataset = dataset.batch(params["batch_size"], drop_remainder=True)

In [35]:
from seq2seq.layers import Encoder, BahdanauAttention, Decoder

## 构建模型

In [31]:
encoder = Encoder(params["vocab_size"], params["embed_size"], embedding_matrix, params["enc_units"], params["batch_size"])
enc_hidden = encoder.initialize_hidden_state()
example_input_batch = tf.ones(shape=(params["batch_size"], params["max_enc_len"]), dtype=tf.int32)
sample_output, sample_hidden = encoder(example_input_batch, enc_hidden)
sample_output.shape
sample_hidden.shape

(64, 200, 300)


TensorShape([64, 1024])

In [36]:
decoder = Decoder(params["vocab_size"], params["embed_size"], embedding_matrix, params["enc_units"], params["batch_size"])
sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), sample_hidden, sample_output)
sample_decoder_output.shape

TensorShape([64, 32909])

## 保存点设置

In [49]:
checkpoint_dir = "./../data/checkpoints/training_checkpoints_mask_loss_dim300_seq"
import os
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

## 训练

In [42]:
optimizer = tf.keras.optimizers.Adam(name='Adam',learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


pad_index=vocab['<PAD>']
nuk_index=vocab['<UNK>']

def loss_function(real, pred):
    pad_mask = tf.math.equal(real, pad_index)
    nuk_mask = tf.math.equal(real, nuk_index)
    mask = tf.math.logical_not(tf.math.logical_or(pad_mask,nuk_mask))
    
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [43]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        # 1. 构建encoder
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        # 2. 复制
        dec_hidden = enc_hidden
        # 3. <START> * BATCH_SIZE 
        dec_input = tf.expand_dims([vocab['<START>']] * params["batch_size"], 1)
        
        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # decoder(x, hidden, enc_output)
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            
            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))

        variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss

In [ ]:
import time
epochs = params["epochs"]
# 如果检查点存在，则恢复最新的检查点。
if checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir)):
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    print ('Latest checkpoint restored!!')
    
for epoch in range(epochs):
    start = time.time()
    total_loss = 0
    enc_hidden = encoder.initialize_hidden_state()

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 1 == 0:
            print('Epoch {} Step {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             checkpoint_prefix))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Latest checkpoint restored!!
Epoch 1 Step 0 Loss 2.2436
Epoch 1 Step 1 Loss 2.4755
Epoch 1 Step 2 Loss 2.3152
Epoch 1 Step 3 Loss 2.0714
Epoch 1 Step 4 Loss 2.0473
Epoch 1 Step 5 Loss 1.9664
Epoch 1 Step 6 Loss 1.3815
Epoch 1 Step 7 Loss 1.5031
Epoch 1 Step 8 Loss 1.1857
Epoch 1 Step 9 Loss 1.8654
Epoch 1 Step 10 Loss 1.8284
Epoch 1 Step 11 Loss 1.7517
Epoch 1 Step 12 Loss 1.5243
Epoch 1 Step 13 Loss 1.8855
Epoch 1 Step 14 Loss 1.7766
Epoch 1 Step 15 Loss 1.9253
Epoch 1 Step 16 Loss 1.5869
Epoch 1 Step 17 Loss 1.3390
Epoch 1 Step 18 Loss 0.9561
Epoch 1 Step 19 Loss 1.0416
Epoch 1 Step 20 Loss 1.2915
Epoch 1 Step 21 Loss 1.3680
Epoch 1 Step 22 Loss 1.4378
Epoch 1 Step 23 Loss 2.0461
Epoch 1 Step 24 Loss 1.4714
Epoch 1 Step 25 Loss 1.5752
Epoch 1 Step 26 Loss 1.5738
Epoch 1 Step 27 Loss 2.0820
Epoch 1 Step 28 Loss 2.2571
Epoch 1 Step 29 Loss 1.9164
Epoch 1 Step 30 Loss 1.8014
Epoch 1 Step 31 Loss 1.7590
Epoch 1 Step 32 Loss 1.7748
Epoch 1 Step 33 Loss 1.3229
Epoch 1 Step 34 Loss 1.5937
E

Epoch 1 Step 286 Loss 1.4635
Epoch 1 Step 287 Loss 1.6524
Epoch 1 Step 288 Loss 1.9989
Epoch 1 Step 289 Loss 1.8142
Epoch 1 Step 290 Loss 1.9594
Epoch 1 Step 291 Loss 1.3868
Epoch 1 Step 292 Loss 1.5491
Epoch 1 Step 293 Loss 1.6022
Epoch 1 Step 294 Loss 1.5234
Epoch 1 Step 295 Loss 1.7511
Epoch 1 Step 296 Loss 1.9255
Epoch 1 Step 297 Loss 1.7385
Epoch 1 Step 298 Loss 1.7829
Epoch 1 Step 299 Loss 1.6567
Epoch 1 Step 300 Loss 1.7034
Epoch 1 Step 301 Loss 1.8462
Epoch 1 Step 302 Loss 1.7274
Epoch 1 Step 303 Loss 1.9971
Epoch 1 Step 304 Loss 1.6847
Epoch 1 Step 305 Loss 1.5394
Epoch 1 Step 306 Loss 1.8865
Epoch 1 Step 307 Loss 1.5472
Epoch 1 Step 308 Loss 1.6995
Epoch 1 Step 309 Loss 1.7516
Epoch 1 Step 310 Loss 1.4110
Epoch 1 Step 311 Loss 1.5036
Epoch 1 Step 312 Loss 1.4445
Epoch 1 Step 313 Loss 1.6686
Epoch 1 Step 314 Loss 1.8867
Epoch 1 Step 315 Loss 2.0305
Epoch 1 Step 316 Loss 1.7440
Epoch 1 Step 317 Loss 1.6946
Epoch 1 Step 318 Loss 1.7450
Epoch 1 Step 319 Loss 1.6756
Epoch 1 Step 3

Epoch 1 Step 569 Loss 1.9687
Epoch 1 Step 570 Loss 1.8417
Epoch 1 Step 571 Loss 1.6521
Epoch 1 Step 572 Loss 1.4176
Epoch 1 Step 573 Loss 1.2709
Epoch 1 Step 574 Loss 1.6076
Epoch 1 Step 575 Loss 1.7721
Epoch 1 Step 576 Loss 1.6970
Epoch 1 Step 577 Loss 1.8982
Epoch 1 Step 578 Loss 1.8142
Epoch 1 Step 579 Loss 1.7504
Epoch 1 Step 580 Loss 1.6760
Epoch 1 Step 581 Loss 1.5898
Epoch 1 Step 582 Loss 1.6699
Epoch 1 Step 583 Loss 1.8077
Epoch 1 Step 584 Loss 1.5253
Epoch 1 Step 585 Loss 1.3114
Epoch 1 Step 586 Loss 1.4344
Epoch 1 Step 587 Loss 1.3075
Epoch 1 Step 588 Loss 1.2855
Epoch 1 Step 589 Loss 1.3504
Epoch 1 Step 590 Loss 1.4149
Epoch 1 Step 591 Loss 1.8048
Epoch 1 Step 592 Loss 1.7571
Epoch 1 Step 593 Loss 1.6703
Epoch 1 Step 594 Loss 1.8602
Epoch 1 Step 595 Loss 2.0419
Epoch 1 Step 596 Loss 1.7697
Epoch 1 Step 597 Loss 1.7983
Epoch 1 Step 598 Loss 1.4311
Epoch 1 Step 599 Loss 1.3124
Epoch 1 Step 600 Loss 1.7091
Epoch 1 Step 601 Loss 1.8946
Epoch 1 Step 602 Loss 1.2047
Epoch 1 Step 6

## 预测

In [ ]:
def evaluate(model, inputs):
    attentions_plot = np.zeros((max_length_targ, max_length_inp))
    inputs = tf.convert_to_tensor(inputs)
    
    result = ""
    
    hidden = [tf.zeros((1, units))]
    enc_output, enc_hidden = model.encoder(inputs, hidden)
    
    dec_hidden = enc_hidden
    
    dec_input = tf.expand_dims([vocab["<START>"]], 0)
    
    context_vectors, _ = model.attention(dec_hidden, enc_output)
    
    for t in range(max_length_targ):
        context_vector, attention_weights = model.attention(dec_hidden, enc_output)

        predictions. dec_hidden = model.decoder(dec_input, dec_hidden, enc_output, context_vector)
        
        attention_weights = tf.reshape(attention_weights, (-1, ))
        
        attention_plot[t] = attention_weights.numpy()
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        result += reverse_vocab[predicted_id] + " "
        if reverse_vocab[predicted_id] == "<STOP>":
            return result, sentence, attention_plot 
        
        dec_input = tf.expand_dims([predicted_id], 0)
        
    return result, sentence, attention_plot

In [52]:
def translate(sentence):
    sentence = procepross_sentence(sentence, max_length_inp, vocab)
    
    result, sentence, attention_plot = evaluate(model, sentence)
    
    print("input: %s" % (sentence))
    print("predicted translation: {}".format(result))
    
    attention_plot = attention_plot[:len(result.split(" ")), :len(sentence.split(" "))]
    plot_attention(attention_plot, sentence.split(" "), result.split(" "))